# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

   
## Описание данных датасета
    
    
- *children* — количество детей в семье

- *days_employed* — общий трудовой стаж в днях

- *dob_years* — возраст клиента в годах

- *education* — уровень образования клиента

- *education_id* — идентификатор уровня образования

- *family_status* — семейное положение

- *family_status_id* — идентификатор семейного положения

- *gender* — пол клиента

- *income_type* — тип занятости

- *debt* — имел ли задолженность по возврату кредитов

- *total_income* — ежемесячный доход

- *purpose* — цель получения кредита

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [39]:
# Для считывания таблицы csv импортируем библиотеку Pandas 

import pandas as pd

# Для лемматизации импортируем функцию Mystem из библиотеки pymystem3

from pymystem3 import Mystem

In [40]:
# Применям метод read_csv и сохраним в переменной data
data = pd.read_csv('data.csv')

# для хранения функции Mystem() создаем переменную m

m = Mystem()

# Общую информацию датафрейма получим с помощью метода .info()
data.info()
# Показать таблицу
data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


**Вывод**

В таблице 21525 строк и 12 столбцов. 
Названия столбцов соответствуют "правильной" стилистике:

-змеиный_регистр

-в названии применяются строчные латинские буквы

Следовательно, переименование столбцов таблицы не требуется.

В таблице присутствуют различные типы данных: целочисленный (_int_), тип данных с плавающей точкой (_float_) и типа _object_ (_str_ и другие).

## Шаг 2. Предобработка данных

Можно заметить, что в столбцах **days_employed** и **total_income** отсутствуют данные, в связи с чем требуется предобратка данных по пропускам перед исследованием.

Также можно заметить некорректность данных в **days_employed**. По определению "Общий трудовой стаж в днях" - это суммарная продолжительность рабочей деятельности...то есть значение этого показателя неотрицательно (>=0). Кроме этого, некоторые значения слишком большие (340266 дней равносильно 932 годам).

In [41]:
# В столбцах education и family_status приведем все значения к нижнему регистру
data['education'] = data['education'].str.lower()
data['family_status'] = data['family_status'].str.lower()

### Обработка пропусков

***Определите и заполните пропущенные значения***

***Опишите, какие пропущенные значения вы обнаружили***

In [42]:
# Подсчет количества пропусков в таблице
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

или

In [43]:
# Количество пропусков в столбцах days_employed и total_income
print(len(data[data['total_income'].isna()]))
print(len(data[data['days_employed'].isna()]))

2174
2174


In [44]:

# Количество пропусков в столбцах days_employed и total_income одинаково, 
# проверим находятся ли они в одинаковых строках или нет 

data.loc[(data['total_income'].isna())&(data['days_employed'].isna())].shape[0]

2174

**Вывод**

В столбцах **days_employed** и **total_income** отсутсвуют данные в 2174 строках, что составляет 10% от всех строк. Поэтому их удаление может сильно исказить результаты исследования. 


***Приведите возможные причины появления пропусков в данных***

Вероятнее всего, пропуски вызваны тем, что клиенты не представили данную информацию банку. Не исключено, что часть данных могла "потеряться" или "удалиться" из-за обработки таблицы работником банка. 

В любом случае, пропуски есть и их отсутсвие нужно каким-то образом исправить так, чтобы наше исследовние было "правдоподобным" и не искаженным.

***Объясните, по какому принципу заполнены пропуски***

При заполнении пропусков в столбце **total_income** прежде всего зададимся вопросом: "А что влияет на ежемесячный доход?". Логичнее всего, что таким определяющим фактором является род деятельности, который представлен в столбце **income_type**. Поэтому, в зависимости от значения в столбце **income_type**, заполним пропуск в столбце **total_income** средним или медианным значением.

In [45]:
data.groupby('income_type')['total_income'].count()

income_type
безработный            2
в декрете              1
госслужащий         1312
компаньон           4577
пенсионер           3443
предприниматель        1
сотрудник          10014
студент                1
Name: total_income, dtype: int64

In [46]:
# Сгруппируем данные по столбцу income_type 
# и посчитаем среднее значение и медианное значения ежемесячного дохода total_income 
# для представления среднего и медианного значений в таблице применим метод agg для столбца total_income
display(data.groupby('income_type').agg({'total_income':['mean','median']}))

total_income               
                          mean         median
income_type                                  
безработный      131339.751676  131339.751676
в декрете         53829.130729   53829.130729
госслужащий      170898.309923  150447.935283
компаньон        202417.461462  172357.950966
пенсионер        137127.465690  118514.486412
предприниматель  499163.144947  499163.144947
сотрудник        161380.260488  142594.396847
студент           98201.625314   98201.625314

Заметим, что некоторые группы встречаются в таблице намного чаще, в связи с чем заметны отклонения ~ 10% между средним и медианным значением дохода. 
Заменим пропуски в столбце **total_income** на медианные значения (тем самым учитывая доход среднестатистического клиента). А вот пропуски в столбце **days_employed** на мой взгляд, логичнее, заполнить средним значениями.

In [47]:
# Создаем переменную для хранения Series() и 
# создания связи "род деятельности"-"медианное значение месячного дохода для этой категории" и "род деятельности"-"среднее значение количества рабочих дней"
total_income_median = data.groupby('income_type')['total_income'].median()
days_employed_mean = data.groupby('dob_years')['days_employed'].mean()

In [48]:
# Добавляем столбец, в котором будут храниться средние значения.
data['median'] = data.groupby('income_type')['total_income'].transform('median')
data['mean'] =  data.groupby('dob_years')['days_employed'].transform('mean')
# Заполняем пропущенные значения средними.
data['total_income'] = data['total_income'].fillna(data['median'])
data['days_employed'] = data['days_employed'].fillna(data['mean'])



In [49]:
# Убеждаемся, что пропуски отсутствуют
# Логичнее использовать более логическое сложение условий
data['total_income'].loc[(data['total_income'].isna())^(data['days_employed'].isna())].count()

0

In [50]:
# Покажем первые 5 и последние 5 строк таблицы, чтобы убедиться, что пропуски заполнились правильными значениями
display(data.head(), data.tail())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,median,mean
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,142594.396847,3393.631090
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,142594.396847,-21.296136
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,142594.396847,-481.794213
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,142594.396847,427.651495
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,118514.486412,89171.803901


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,median,mean
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем,172357.950966,4530.036926
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем,118514.486412,316720.285499
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость,142594.396847,3095.214328
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля,142594.396847,3095.214328
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля,142594.396847,2446.420996


Как можно заметить, замена правильно подставила медианные значения доходов в соответствии с родом деятельности и средние значения рабочих дней в соответствии с возрастом клиентов.

Пропуски отсутсвтуют, а значит, можно приступать к следующему пункту предобработки.

### Замена типа данных

***Замените вещественный тип данных на целочисленный***

В таблице тип данных (*float*) имеют столбцы **days_employed** и **total_income**. Для изменения типа данных применим метод astype().

In [51]:
data['total_income'] = data['total_income'].astype(int)
data['days_employed'] = data['days_employed'].astype(int)
data.dtypes

children              int64
days_employed         int64
dob_years             int64
education            object
education_id          int64
family_status        object
family_status_id      int64
gender               object
income_type          object
debt                  int64
total_income          int64
purpose              object
median              float64
mean                float64
dtype: object

Теперь у все числовые данные у нас целочисленные (*int*).

**Вывод**

***Поясните, как выбирали метод для изменения типа данных***

При переводе числе из вещественного типа в целочисленный достаточно применить метод astype().

В случае, если бы какие-то числовые данные были представлены в виде строки (типа *object*), то для начала нужно было провести перевод в вещественный тип с помощью функции to_numeric() и только потом применить метод astype().

### Обработка дубликатов

Посчитаем количество явных строк-дубликатов.

In [52]:
data.duplicated().sum()

71

Выявлено 71 явных дубликатов. Для их исключения воспользуемся методом drop_duplicates() с обновлением индексов

In [53]:
data = data.drop_duplicates().reset_index(drop = True)
# Узнаем количество строк в обновленной таблице
data.shape[0]

21454

Количество строк таблицы изменилось с 21525 до 21454, то есть уменьшилось на количество явных дубликатов.

Далее попробуем отыскать неявные дубликаты или данные с некорректными данными. Можно воспользоваться методом unique(), но мне также интересно количество таких уникальных значений, поэтому воспользуюсь методом value_counts()

In [54]:
for row in data.columns:
    print(f'Значения столбца {row} - {data[row].sort_values().unique()}')
#При подсчете частоты упоминаний того или иного параметра воспользуемся методом sort_index() с параметром ascending равным True, для сортировки значений по порядку.    
    print(data[row].value_counts().sort_index(ascending = True))
    print('_____________________________________')

Значения столбца children - [-1  0  1  2  3  4  5 20]
-1        47
 0     14091
 1      4808
 2      2052
 3       330
 4        41
 5         9
 20       76
Name: children, dtype: int64
_____________________________________
Значения столбца days_employed - [-18388 -17615 -16593 ... 401675 401715 401755]
-18388     1
-17615     1
-16593     1
-16264     1
-16119     1
          ..
 401663    1
 401674    1
 401675    1
 401715    1
 401755    1
Name: days_employed, Length: 9130, dtype: int64
_____________________________________
Значения столбца dob_years - [ 0 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41
 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65
 66 67 68 69 70 71 72 73 74 75]
0     101
19     14
20     51
21    111
22    183
23    252
24    264
25    357
26    408
27    493
28    503
29    544
30    537
31    559
32    509
33    581
34    601
35    616
36    554
37    536
38    597
39    572
40    607
41    605
42    596
43    51

Из выгрузки видны следующие ошибки:

- в столбце ***children*** отрицательное значение -1 (скорее всего ошибка при заполнении, нужно было 1). Также очень сильно выбивается значение 20 (скорее всего ошибка при заполнении, нужно было 2).

- в столбце ***gender*** отсутствует 1 значение, которое обозначено как 'XNA' (принципиальное значение этого параметра для нас не интересует, в исследовании зависимости по половому признаку не требуется. Можно исключить эту строку, можно определить ему любое из значений из F и M, применяя метод replace()).

- в столбце ***dob_years*** присутствуют нулевые значения (принципиальное значение этого параметра для нас не интересует, но их количество больше, чем количество ошибок в столбце ***gender***, их обработка потребует чуть больше усилий нежели прошлый пункт). 

Однако, множество столбцов позволило нам найти и отсеять явные дубликаты. А значит для нашего исследования можно исключить следующие столбцы: ***days_employed***, ***gender***, ***dob_years***, ***education***, ***education_id***, ***family_status_id***.
Удаление лишних столбцов позволит упростить таблицу для нашего исследования и позволит исключить предобработку незначимых для нашего исследований данных, оставив **самое важное**.

**Примечание:** К слову,отбросить столбец ***days_employed*** изначально. Наличие значений этого столбца не влияло на выявления явных дубликатов, но влияло на сложность предобработки пропусков.





In [55]:
data['children'] = data['children'].replace(-1,1)
data['children'] = data['children'].replace(20,2)
'''Оставим в нашей таблице самые необходимые столбцы'''
data = data[['children', 'family_status', 'income_type', 'debt', 'total_income', 'purpose']]

In [56]:
for row in data.columns:
    print(f'Значения столбца {row} - {data[row].sort_values().unique()}')
    print('')
    print('__________')

Значения столбца children - [0 1 2 3 4 5]

__________
Значения столбца family_status - ['в разводе' 'вдовец / вдова' 'гражданский брак' 'женат / замужем'
 'не женат / не замужем']

__________
Значения столбца income_type - ['безработный' 'в декрете' 'госслужащий' 'компаньон' 'пенсионер'
 'предприниматель' 'сотрудник' 'студент']

__________
Значения столбца debt - [0 1]

__________
Значения столбца total_income - [  20667   21205   21367 ... 1726276 2200852 2265604]

__________
Значения столбца purpose - ['автомобили' 'автомобиль' 'высшее образование'
 'дополнительное образование' 'жилье' 'заняться высшим образованием'
 'заняться образованием' 'на покупку автомобиля'
 'на покупку подержанного автомобиля' 'на покупку своего автомобиля'
 'на проведение свадьбы' 'недвижимость' 'образование' 'операции с жильем'
 'операции с коммерческой недвижимостью' 'операции с недвижимостью'
 'операции со своей недвижимостью' 'покупка жилой недвижимости'
 'покупка жилья' 'покупка жилья для сдачи' 'покупк

**Вывод**

На данном этапе мы почистили таблицу от явных дубликатов, используя метод drop_duplicates(), а также выявили неявные недочеты. Возможно, фунцкия unique() показала бы другой результат, если бы ранее не проводилось приведение некоторых столбцов к низшему регистру. Также обнаружилось, что в данные закрались некорректные данные, которые были заменены методом replace(). Также были удалены не нужные для нашего исследдования. 

### Лемматизация

***Выделите леммы в значениях столбца с целями получения кредита***

Посчитаем количество уникальных записей в столбце ***purpose*** с помощью функции nunique()

In [57]:
data['purpose'].nunique()

38

38 различных значений, при этом можно выделить несколько: *образование*, *недвижимость*, *автомобиль*, *свадьба*.

Для структурирования столбца **purpose** приведем имеющиеся значения к выдвинутым выше категориям.

***Опишите, как вы проводили лемматизацию целей кредита***

In [58]:
# Для приведения целей к какому-то одному виду пропишем функцию
def lem(purpose):
    purpose_difinition = ['образование', 'недвижимость', 'автомобиль', 'свадьба']
    lemmas = m.lemmatize(purpose)
    for row in purpose_difinition:
        if row in lemmas:
            return row
# цели, связанные с недвижимостью, также упоминаются в puprose словом "жилье" и этим словом с другим окончанием, 
# поэтому добавим еще условие по поиску этого слова и возвращаемого при его нахождении "недвижимость"      
        elif 'жилье' in lemmas:
            return 'недвижимость'
        
# Создадим новый столбец purpose_group и сохраним значения 
data['purpose_group'] = data['purpose'].apply(lem)

print('Количество пропусков в столбце purpose_group', data['purpose_group'].isna().sum())

# Проверим, правильно ли заполнились значения в purpose_group в соответствии со значением purpose в столбце
display(data[['purpose','purpose_group']])
        

Количество пропусков в столбце purpose_group 0


,purpose,purpose_group
0,покупка жилья,недвижимость
1,приобретение автомобиля,автомобиль
2,покупка жилья,недвижимость
3,дополнительное образование,образование
4,сыграть свадьбу,свадьба
...,...,...
21449,операции с жильем,недвижимость
21450,сделка с автомобилем,автомобиль
21451,недвижимость,недвижимость
21452,на покупку своего автомобиля,автомобиль


**Вывод**

Все строки в столбце **purpose_group** были заполнены значениями 4-х категорий в соответствии со значением в столбце **purpose**

### Категоризация данных

Исходя из заданных целей исследования структурируем данные по следующим категориям:

- по наличию детей (есть дети или нет детей);

- по уровню заработка (низкий, средний, высокий, очень высокий)

- по целям 



In [59]:
# Создадим функцию для категоризации значений в столбце children

def children_category(row):
    if row['children'] == 0:
        return 'Нет детей'
    else: 
        return 'Есть дети'

# Также добавим функцию по возвращению кагории задолженности (есть или нет), которая пригодится нам для создания столбца debt_status 
# можно уйти от создания функции и прописать обычную замену значения в ячейке через replace

def debt_name(row):
    if row['debt'] == 0:
        return 'отсутствие задолженности'
    else:
        return 'наличие задолженности'
    


In [60]:
# Создадим функцию для категоризации значений в столбце total_income

# Для этого разделим датасет на 4 категории так, чтобы количество строк в каждой категории было примерно одинаковым

pd.qcut(data['total_income'], 4)

0        (195820.25, 2265604.0]
1          (107623.0, 142594.0]
2         (142594.0, 195820.25]
3        (195820.25, 2265604.0]
4         (142594.0, 195820.25]
                  ...          
21449    (195820.25, 2265604.0]
21450     (142594.0, 195820.25]
21451     (20666.999, 107623.0]
21452    (195820.25, 2265604.0]
21453     (20666.999, 107623.0]
Name: total_income, Length: 21454, dtype: category
Categories (4, interval[float64, right]): [(20666.999, 107623.0] < (107623.0, 142594.0] < (142594.0, 195820.25] < (195820.25, 2265604.0]]

Итого у нас 4 категории:
- меньше 107620

- от 107620 до 142594

- от 142594 до 195818

- от 195818

In [61]:
pd.qcut(data['total_income'], 4)

def income_category(row):
    if row['total_income'] < 107620:
        return 'Очень низкий заработок'
    elif row['total_income'] < 142594:
        return 'Низкий заработок'
    elif row['total_income'] < 195818:
        return 'Средний заработок'
    else: 
        return 'Высокий заработок'

In [62]:
# Добавим в таблицу столбцы children_group, debt_status и income_group с названием категории

data['children_group'] = data.apply(children_category, axis = 1)
data['debt_status'] = data.apply(debt_name, axis = 1)
data['income_group'] = data.apply(income_category, axis = 1)

data[['children_group','debt_status','income_group']].head(20)


,children_group,debt_status,income_group
0,Есть дети,отсутствие задолженности,Высокий заработок
1,Есть дети,отсутствие задолженности,Низкий заработок
2,Нет детей,отсутствие задолженности,Средний заработок
3,Есть дети,отсутствие задолженности,Высокий заработок
4,Нет детей,отсутствие задолженности,Средний заработок
5,Нет детей,отсутствие задолженности,Высокий заработок
6,Нет детей,отсутствие задолженности,Высокий заработок
7,Нет детей,отсутствие задолженности,Низкий заработок
8,Есть дети,отсутствие задолженности,Очень низкий заработок
9,Нет детей,отсутствие задолженности,Средний заработок


**Вывод**

В таблице созданы столбцы, которые создержат значение категории по наличиюю детей, уровню заработка или целевому значению кредита.

Данные таблицы обработаны и готовы к исследованию

## Шаг 3. Ответьте на вопросы

Для ответов на поставленные вопросы будем создавать сводные таблицы с помощью функции pivot_table, содержащие необходимые данные. 
Также, добавим столбец с расчетом доли должников по каждой категории, значение которого будет определяющим в ответе на поставленный вопрос

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [63]:
children_pivot = data.pivot_table(
    index = 'children_group',   
    columns = 'debt_status',
    values = 'debt', 
    aggfunc = 'count')
children_pivot['total'] = children_pivot['отсутствие задолженности']+children_pivot['наличие задолженности']
children_pivot['debtor_percent'] = children_pivot['наличие задолженности'] * 100 / (children_pivot['отсутствие задолженности']+children_pivot['наличие задолженности'])

In [64]:
children_pivot.sort_values(by = 'debtor_percent')

debt_status,наличие задолженности,отсутствие задолженности,total,debtor_percent
children_group,,,,
Нет детей,1063,13028,14091,7.543822
Есть дети,678,6685,7363,9.208203


**Вывод**

Наличие задолженности по кредиту присуще как для семей с детьми, так и для семей без детей.
Несмотря на большее количество кредитов у семей без детей, доля должников среди них ниже, чем у другой группы людей.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [65]:
family_status_pivot = data.pivot_table(
    index = 'family_status',   
    columns = 'debt_status',
    values = 'debt', 
    aggfunc = 'count'
    )

family_status_pivot['total'] = family_status_pivot['отсутствие задолженности']+family_status_pivot['наличие задолженности']

family_status_pivot['debtor_percent'] = family_status_pivot['наличие задолженности'] * 100 / \
    (family_status_pivot['отсутствие задолженности']+family_status_pivot['наличие задолженности'])

In [66]:
family_status_pivot.sort_values(by = 'debtor_percent')

debt_status,наличие задолженности,отсутствие задолженности,total,debtor_percent
family_status,,,,
вдовец / вдова,63,896,959,6.569343
в разводе,85,1110,1195,7.112971
женат / замужем,931,11408,12339,7.545182
гражданский брак,388,3763,4151,9.347145
не женат / не замужем,274,2536,2810,9.750890


**Вывод**

Большее значение числа кредитов присуще для женатых и замужних людей.
Как видно из показателей последнего столбца, доля должников среди вдовцов/вдов наименьшая в выборке. Высокие значения показателя присущи для граждан в гражданском браке и не женатых / не замужних.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [67]:
income_pivot = data.pivot_table(
    index = 'income_group',   
    columns = 'debt_status',
    values = 'debt', 
    aggfunc = 'count'
    )
    
income_pivot['total'] = income_pivot['отсутствие задолженности']+income_pivot['наличие задолженности']

income_pivot['debtor_percent'] = income_pivot['наличие задолженности'] * 100 / \
 (income_pivot['отсутствие задолженности']+income_pivot['наличие задолженности'])

In [68]:
income_pivot.sort_values(by = 'debtor_percent')

debt_status,наличие задолженности,отсутствие задолженности,total,debtor_percent
income_group,,,,
Высокий заработок,383,4982,5365,7.138863
Очень низкий заработок,427,4936,5363,7.961962
Средний заработок,546,5770,6316,8.644712
Низкий заработок,385,4025,4410,8.730159


**Вывод**

Несмотря на небольшое значение количества кредитов, доля задолжников у людей с низким заработков наибольшая среди всех.
Очень большое количество людей, являющиеся клиентами банка, имеют заработок выше 110000, при этом доля должников среди них составляет чуть выше 8%.
Низкая задолженность присуща для людей со средним заработком.

**Примечание:** Показатель численно может изменяться в зависимости от категоризации значения **income_total**


- Как разные цели кредита влияют на его возврат в срок?

In [69]:
purpose_pivot = data.pivot_table(
    index = 'purpose_group',   
    columns = 'debt_status',
    values = 'debt', 
    aggfunc = 'count')
purpose_pivot['total'] = purpose_pivot['отсутствие задолженности']+purpose_pivot['наличие задолженности']
purpose_pivot['debtor_percent'] = purpose_pivot['наличие задолженности'] * 100 / (purpose_pivot['отсутствие задолженности']+purpose_pivot['наличие задолженности'])

In [70]:
purpose_pivot.sort_values(by = 'debtor_percent')

debt_status,наличие задолженности,отсутствие задолженности,total,debtor_percent
purpose_group,,,,
недвижимость,782,10029,10811,7.233373
свадьба,186,2138,2324,8.003442
образование,370,3643,4013,9.220035
автомобиль,403,3903,4306,9.359034


**Вывод**

Наибольшее количество клиентов банка хотели приобрести недвижимость. При этом, в большинстве случаев, клиенты этой целевой группы вовремя рассчитывались по кредиту.
Основными должниками по кредиту являются клиенты, которые хотели приобрести автомобиль или взяли кредит на обучение (их количество и доля должников примерно одинакова)

## Шаг 4. Общий вывод

В предложенной таблице 21525 строк и 12 столбцов. Названия столбцов соответствали "правильной" стилистике, поэтому переименование столбцов таблицы не производилось.

В таблице присутствуют различные типы данных: целочисленный (int), тип данных с плавающей точкой (float) и типа object (str и другие).

В столбцах **days_employed** и **total_income** отсутствовали данные, в связи с чем требовалась предобратка данных по пропускам перед исследованием.

**Примечание:** часть данных, можно было исключить заранее, для упрощения предобработки, так как их наличие никак не влияло на результаты предобработки, но их наличие требовало в том или ином месте обработки (что немного усложняло код и увеличивало время предобработки).

В столбцах **days_employed** и **total_income** отсутсвли данные в 2174 строках, что составляло 10% от всех строк. Поэтому их удаление могло сильно исказить результаты исследования. Вероятнее всего, пропуски вызваны тем, что клиенты не представили данную информацию банку. Не исключено, что часть данных могла "потеряться" или "удалиться" из-за обработки таблицы работником банка.

При заполнении пропусков в столбце **total_income** предполагалось, что определяющим фактором являлся род деятельности, который представлен в столбце **income_type**. В зависимости от значения в столбце **income_type**, пропуск в столбце **total_income** заполнен медианным значением. 

Пропуски в столбце **days_employed** заполнены средним значениями (сгруппированным по возрасту клиентов **dob_years** . Вместе с этим была праизведена замена отрицательных значений этого столбца, путем умножения на (-1).

Следующим шагом была замена вещественных числовых данных в таблице. Тип данных (*float*) имели столбцы **days_employed** и **total_income**. Для изменения типа данных использовали метод astype().

Следующим шагом была предобработка дубликатов. Было выявлено 71 явных дубликатов. Для их исключения воспользуемся методом drop_duplicates() с обновлением индексов.

Количество строк таблицы изменилось с 21525 до 21454, то есть уменьшилось на количество явных дубликатов.

Неявные дубликаты или данные с некорректными данными найдены с помощью метода unique(), а для определения частоты таких уникальных значений, применен метод value_counts(). Были выявлены следующие ошибки:

- в столбце ***children*** отрицательное значение -1 (скорее всего ошибка при заполнении, нужно было 1). Также очень сильно выбивается значение 20 (скорее всего ошибка при заполнении, нужно было 2).

- в столбце ***gender*** отсутствует 1 значение, которое обозначено как 'XNA' (принципиальное значение этого параметра для нас не интересует, в исследовании зависимости по половому признаку не требуется. Можно исключить эту строку, можно определить ему любое из значений из F и M, применяя метод replace()).

- в столбце ***dob_years*** присутствуют нулевые значения (принципиальное значение этого параметра для нас не интересует, но их количество больше, чем количество ошибок в столбце ***gender***, их обработка потребует чуть больше усилий нежели прошлый пункт). 

Однако, множество столбцов позволило нам найти и отсеять явные дубликаты. А значит для нашего исследования можно исключить следующие столбцы: ***days_employed***, ***gender***, ***dob_years***, ***education***, ***education_id***, ***family_status_id***.
Удаление лишних столбцов позволит упростить таблицу для нашего исследования и позволит исключить предобработку незначимых для нашего исследований данных, оставив **самое важное**.

В шаге лемматизация было обнаружено, что столбец **purpose** имел 38 различных значений, при этом среди них можно было выделить несколько: *образование*, *недвижимость*, *автомобиль*, *свадьба*. Результаты лемматизации сведены в новый столбец **purpose_group**.



***Есть ли зависимость между наличием детей и возвратом кредита в срок***

Исследование показало, что клиенты без детей чаще возвращают кредит в срок. Процент должников - 7,54% (против значения 9,21% для клиентов с детьми)


***Есть ли зависимость между семейным положением и возвратом кредита в срок***

Исследование показало, что основыми клиентами являются семейные люди (женатые / замужем). Процент клиентов этой категории, возвращающих кредит не в срок равен 7,54%.
Чаще всего задерживают выплату по кредитам холостые люди или люди, нахоядщиеся в гражданском браке (9,75% и 9,35% соответственно).
Люди в разводе или потерявших своих близких реже берут кредиты, но при этом чаще стараются выплачивают кредиты в срок (доли  должников равны 7,11% и 6,56% соответственно).


***Есть ли зависимость между уровнем дохода и возвратом кредита в срок***

Исследование показало, что клиенты заработок которых находится в диапазоне 107620..142594 чаще всего возвращают кредит не в срок (процент должников составляет 8,73%).

Люди со средним заработком чаще всех берут кредиты. Процент должников сотавляет 8,64%. 

Клиенты с заработком выше 195819 и с заработком ниже 107620 чаще возвращают кредит в срок. Процент должников 7,16% и 7,96% соответственно.


***Как разные цели кредита влияют на его возврат в срок***

Исследование показало, что чаще всего берут кредиты для покупки недвижимости. При этом, клиенты с этой целью чаще всего платили в срок (процент должников составляет 7,23%).

Чаще всего задерживают выплаты по кредитам, взятые на получение образования или покупку автомобился (9,22% и 9,36%).

Реже всего берут кредиты на организацию свадеб. Процент должников этой целевой категории 8,00%.








